# Demo JetBot with Road Following, Collision Avoidance & Sign Handling

This small demo represents an object oriented solution for combining road following, collision avoidance and sign detection models to showcase the driving capabilities of the JetBot.

For optimal usage, adjust the camera angle to ~55° and consider proper lighting conditions without reflections.

1. Add Imports

In [1]:
from libjetbot.ExtendedRobot import ExtendedRobot, HandleTypes
from libjetbot.handles.ObjectDetectionHandle import ObjectDetectionHandle
from libjetbot.handles.RoadFollowingHandle import RoadFollowingHandle
from libjetbot.handles.CollisionDetectionHandle import CollisionDetectionHandle

import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

2. Initialize the camera

Add widgets to visualize camera image and additional robot information

In [2]:
camera = Camera.instance(width=224, height=224, fps=10)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

last_detected = widgets.Text(value='', placeholder='no sign detected yet', description='Last detected sign:', disabled=False)
sss = widgets.IntText(value=0, placeholder='', description='Stop STATE', disabled=False)
fcounter = widgets.IntText(value=0, placeholder='', description='Frame counter', disabled=False)

3. Initialize the robot and the road following- / collision avoidance- / object detection handles

 register them for execution on new image frames

In [3]:
bot = ExtendedRobot(camera)

cdh = CollisionDetectionHandle("JetbotYolo/weights/collision_model_resnet_19_04_a_trt.pth")
rfh = RoadFollowingHandle("JetbotYolo/weights/best_steering_model_xy_17_04_a_trtv2.pth")
odh = ObjectDetectionHandle("JetbotYolo/weights/object_detection_model_trt.pth", trt=True)


bot.register(HandleTypes.COLLISION_DETECTION, cdh)
bot.register(HandleTypes.ROAD_FOLLOWING, rfh)
bot.register(HandleTypes.OBJECT_DETECTION, odh)

construct
init JetsonYoloTest


Model summary: 214 layers, 1770682 parameters, 0 gradients


Registering module 'HandleTypes.COLLISION_DETECTION'
Registering module 'HandleTypes.ROAD_FOLLOWING'
Registering module 'HandleTypes.OBJECT_DETECTION'


4. Create links for the traitlets to show values on change and display them

In [5]:
last_detected_link = traitlets.dlink( (odh, 'last_detected'), (last_detected, 'value'))
sss_link = traitlets.dlink( (odh, 'state_sign_stop'), (sss, 'value'))
fcounter_link = traitlets.dlink( (odh, 'global_fcounter'), (fcounter, 'value'))

#slider_sss = widgets.IntSlider(value=0, min=0, max=3, step=1, description='state sign stop')
#link_sss = traitlets.dlink( (slider_sss, 'value'), (odh, 'state_sign_stop')) 

display(image, last_detected, sss, fcounter)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Text(value='', description='Last detected sign:', placeholder='no sign detected yet')

IntText(value=0, description='Stop STATE')

IntText(value=0, description='Frame counter')

5. Start the bot.

**Caution**: The bot will start driving. **Please hold the bot** and wait until the camera feed shown in the notebook updates before releasing it again. (This process can take a few seconds)

Now it should be able to drive on the pre-trained track and react to the pre-trained signs

In [6]:
bot.start()

start


6. Stop the bot. You may start the bot again.

In [ ]:
bot.pause()

7. Stop and destroy the camera. The camera is now free to be used by other scripts. To re-enable this script, restart the kernel for this script and go to step 1.

In [ ]:
camera.stop()